# Feature Engineering Homework 
***
**Name**: Pranav Kumar Sivakumar 

**Kaggle Username**: $PranavPKS$
***

This assignment is due on Moodle by **5pm on Friday February 23rd**. Additionally, you must make at least one submission to the **Kaggle** competition before it closes at **4:59pm on Friday February 23rd**. Submit only this Jupyter notebook to Moodle. Do not compress it using tar, rar, zip, etc. Your solutions to analysis questions should be done in Markdown directly below the associated question.  Remember that you are encouraged to discuss the problems with your instructors and classmates, but **you must write all code and solutions on your own**.  For a refresher on the course **Collaboration Policy** click [here](https://github.com/chrisketelsen/CSCI5622-Machine-Learning/blob/master/resources/syllabus.md#collaboration-policy)



## Overview 
***

When people are discussing popular media, there’s a concept of spoilers. That is, critical information about the plot of a TV show, book, or movie that “ruins” the experience for people who haven’t read / seen it yet.

The goal of this assignment is to do text classification on forum posts from the website [tvtropes.org](http://tvtropes.org/), to predict whether a post is a spoiler or not. We'll be using the logistic regression classifier provided by sklearn.

Unlike previous assignments, the code provided with this assignment has all of the functionality required. Your job is to make the functionality better by improving the features the code uses for text classification.

**NOTE**: Because the goal of this assignment is feature engineering, not classification algorithms, you may not change the underlying algorithm or it's parameters

This assignment is structured in a way that approximates how classification works in the real world: Features are typically underspecified (or not specified at all). You, the data digger, have to articulate the features you need. You then compete against others to provide useful predictions.

It may seem straightforward, but do not start this at the last minute. There are often many things that go wrong in testing out features, and you'll want to make sure your features work well once you've found them.


## Kaggle In-Class Competition 
***

In addition to turning in this notebook on Moodle, you'll also need to submit your predictions on Kaggle, an online tournament site for machine learning competitions. The competition page can be found here:  

[https://www.kaggle.com/c/feature-engineering-csci-5622-spring-2018](https://www.kaggle.com/c/feature-engineering-csci-5622-spring-2018)

Additionally, a private invite link for the competition has been posted to Piazza. 

The starter code below has a `model_predict` method which produces a two column CSV file that is correctly formatted for Kaggle (predictions.csv). It should have the example Id as the first column and the prediction (`True` or `False`) as the second column. If you change this format your submissions will be scored as zero accuracy on Kaggle. 

**Note**: You may only submit **THREE** predictions to Kaggle per day.  Instead of using the public leaderboard as your sole evaluation processes, it is highly recommended that you perform local evaluation using a validation set or cross-validation. 

In [40]:
import numpy as np
import pandas as pd
#import matplotlib.pylab as plt
#%matplotlib inline 
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import re
import string
import gc
from scipy.sparse import coo_matrix, hstack
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#from nltk.stem.snowball import SnowballStemmer
#from nltk.stem import PorterStemmer
#from sklearn.model_selection import GridSearchCV
from scipy import sparse
wl = WordNetLemmatizer()
sid = SentimentIntensityAnalyzer()
import pickle
from sklearn.feature_selection import SelectFromModel
#stemmer = SnowballStemmer("english")
#ps = PorterStemmer()
#question_mark_RE_str = '\?'
#exclamation_point_RE_str = '\!'
#interrobang_RE_str = '[\?\!]{2,}'
#from nltk.tokenize import word_tokenize

In [ ]:
import json
import http.client
import time
import re

def get_gr(txt):
    conn = http.client.HTTPSConnection("api.themoviedb.org")
    conn.request("GET", "/3/search/tv?page=1&query="+txt+"&language=en-US&api_key=ffdfecf0830968b442b85db5a9cb58a9")
    data = conn.getresponse().read()
    if json.loads(data)['results']:
        id_temp = json.loads(data)['results'][0]['id']
        try:
            conn.request("GET", "/3/tv/"+str(id_temp)+"?language=en-US&api_key=ffdfecf0830968b442b85db5a9cb58a9")
        except:
            conn.request("GET", "/3/tv/"+str(id_temp)+"?language=en-US&api_key=ffdfecf0830968b442b85db5a9cb58a9")
        dd = conn.getresponse().read()
        temp_genre=""
        if json.loads(dd)['genres']:
            for j in range(len(json.loads(dd)['genres'])):
                temp_genre = temp_genre + json.loads(dd)['genres'][j]['name'].replace("& ", "") + " "
            g = temp_genre[:-1]
        else:
            g = ""
        if json.loads(dd)['episode_run_time']:
            r= float(json.loads(dd)['episode_run_time'][0])/100.0
        else:
            r= 0.4
        return g,r
    else:
        return "",0.4
    
def data_col(filepath):
    genres={}
    runtime={}
    df = pd.read_csv(filepath)
    pages = list(set(df["page"]))
    for i in range(len(pages)):
        txt = ' '.join(re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', pages[i])).lower().replace(" ","%20")
        try:
            genres[pages[i]],runtime[pages[i]]=get_gr(txt)
        except:
            #print(i)
            time.sleep(3) 
            genres[pages[i]],runtime[pages[i]]=get_gr(txt)

    gen=[]
    rt=[]
    for i in range(len(df["page"])):
        gen.append(genres[df["page"][i]])
        rt.append(runtime[df["page"][i]])
    return gen,rt
        
gen,rt = data_col("../data/spoilers/train.csv")
gen_test,rt_test = data_col("../data/spoilers/test.csv")

with open("gen.txt", "wb") as fp:
    pickle.dump(gen, fp)
with open("rt.txt", "wb") as fp:
    pickle.dump(rt, fp)
with open("gen_test.txt", "wb") as fp:
    pickle.dump(gen_test, fp)
with open("rt_test.txt", "wb") as fp:
    pickle.dump(rt_test, fp)

In [36]:
with open("gen.txt", "rb") as fp:
    gen = pickle.load(fp)
with open("rt.txt", "rb") as fp:
    rt = pickle.load(fp)
with open("gen_test.txt", "rb") as fp:
    gen_test = pickle.load(fp)
with open("rt_test.txt", "rb") as fp:
    rt_test =  pickle.load(fp)

### [25 points] Problem 1: Feature Engineering 
***

The `FeatEngr` class is where the magic happens.  In it's current form it will read in the training data and vectorize it using simple Bag-of-Words.  It then trains a model and makes predictions.  

25 points of your grade will be generated from your performance on the the classification competition on Kaggle. The performance will be evaluated on accuracy on the held-out test set. Half of the test set is used to evaluate accuracy on the public leaderboard.  The other half of the test set is used to evaluate accuracy on the private leaderboard (which you will not be able to see until the close of the competition). 

You should be able to significantly improve on the baseline system (i.e. the predictions made by the starter code we've provided) as reported by the Kaggle system.  Additionally, the top **THREE** students from the **PRIVATE** leaderboard at the end of the contest will receive 5 extra credit points towards their Problem 1 score.


In [82]:
def preprocess(example):
    if len(re.findall(r'\!', example)) > 0:
        example = example + " PUNCxEXCLAMATIONPOINT"
    if len(re.findall(r'\?', example)) > 0:
        example = example + " PUNCxQUESTIONMARK"
    if len(re.findall(r'[\?\!]{2,}', example)) > 0:
        example = example + " PUNCxINTERROBANG"
    return (' '.join([ wl.lemmatize(word,'v') for word in list(filter(None,re.sub(r'[^\w\s]','',example).lower().split(' ')))]))

class FeatEngr:
    def __init__(self):
        self.vectorizer = CountVectorizer(ngram_range=(2, 3), analyzer='word', stop_words='english',strip_accents='ascii')
        self.tf_transformer = TfidfVectorizer(ngram_range=(1, 3), analyzer='word', preprocessor= preprocess, stop_words='english', norm = 'l2', use_idf=True, smooth_idf= True, strip_accents='ascii')
        self.tf_transformer2 = TfidfVectorizer(analyzer='word',use_idf=True,smooth_idf=True)
        self.tf_transformer3 = TfidfVectorizer(ngram_range=(1, 3), analyzer='word', stop_words='english', norm = 'l2', use_idf=True, smooth_idf= True, strip_accents='ascii')
        self.tf_transformer4 = TfidfVectorizer(analyzer='word',use_idf=True,smooth_idf=True)
        
    def build_train_features(self, examples,tropes,genres):
        """
        Method to take in training text features and do further feature engineering 
        Most of the work in this homework will go here, or in similar functions  
        :param examples: currently just a list of forum posts  
        """
#         for i in range(len(examples)):
#             if len(re.findall(r'\!', examples[i])) > 0:
#                 examples[i] = examples[i] + " PUNCxEXCLAMATION_POINT"
#             if len(re.findall(r'\?', examples[i])) > 0:
#                 examples[i] = examples[i] + " PUNCxQUESTION_MARK"
#             if len(re.findall(r'[\?\!]{2,}', examples[i])) > 0:
#                 examples[i] = examples[i] + " PUNCxINTERROBANG"
#             examples[i] = (' '.join([ wl.lemmatize(word,'v') for word in list(filter(None,re.sub(r'[^\w\s]','',examples[i]).lower().split(' ')))]))

        #return self.tf_transformer.fit_transform(examples)
        allfe = FeatureUnion([("counts", self.vectorizer), ("tfidf", self.tf_transformer),("tfidf2", self.tf_transformer3)]).fit_transform(examples)
        tp = self.tf_transformer2.fit_transform(tropes)
        tp2 = self.tf_transformer4.fit_transform(genres)
        return sparse.csr_matrix(hstack([coo_matrix(allfe),coo_matrix(tp),coo_matrix(tp2)]))

    def get_test_features(self, examples,tropes,genres):
        """
        Method to take in test text features and transform the same way as train features 
        :param examples: currently just a list of forum posts  
        """
#         for i in range(len(examples)):
#             if len(re.findall(r'\!', examples[i])) > 0:
#                 examples[i] = examples[i] + " PUNCxEXCLAMATION_POINT"
#             if len(re.findall(r'\?', examples[i])) > 0:
#                 examples[i] = examples[i] + " PUNCxQUESTION_MARK"
#             if len(re.findall(r'[\?\!]{2,}', examples[i])) > 0:
#                 examples[i] = examples[i] + " PUNCxINTERROBANG"
#             examples[i] = (' '.join([ wl.lemmatize(word,'v') for word in list(filter(None,re.sub(r'[^\w\s]','',examples[i]).lower().split(' ')))]))
        
        #return self.tf_transformer.transform(examples)
        allfe = FeatureUnion([("counts", self.vectorizer), ("tfidf", self.tf_transformer),("tfidf2", self.tf_transformer3)]).transform(examples)
        tp = self.tf_transformer2.transform(tropes)
        tp2 = self.tf_transformer4.transform(genres)
        return sparse.csr_matrix(hstack([coo_matrix(allfe),coo_matrix(tp),coo_matrix(tp2)]))
        

    def show_top10(self):
        """
        prints the top 10 features for the positive class and the 
        top 10 features for the negative class. 
        """
        feature_names = np.asarray(self.vectorizer.get_feature_names())
        top10 = np.argsort(self.logreg.coef_[0])[-10:]
        bottom10 = np.argsort(self.logreg.coef_[0])[:10]
        print("Pos: %s" % " ".join(feature_names[top10]))
        print("Neg: %s" % " ".join(feature_names[bottom10]))
                
    def train_model(self, random_state=1234):
        """
        Method to read in training data from file, and 
        train Logistic Regression classifier. 
        
        :param random_state: seed for random number generator 
        """
        
        from sklearn.linear_model import LogisticRegression 
        
        # load data 
        dfTrain = pd.read_csv("../data/spoilers/train.csv")
        
        # get training features and labels 
        dfTrain["combined"] = dfTrain["sentence"]
        
        senty = np.array([0,0,0,0,0,0])
        for idx,sentence in enumerate(dfTrain["combined"]):
            ss = sid.polarity_scores(sentence)
            temp = np.array([ss['compound'],ss['neg'], ss['neu'], ss['pos'], float(sum(1 for c in sentence if c.isupper()))/float(len(sentence)),float(rt[idx])])
            senty = np.vstack((senty,temp))
        senty = senty[1:]
         
        
        self.X_train = self.build_train_features(list(dfTrain["combined"]),list(dfTrain["trope"]),list(gen))
        
        axx = coo_matrix(self.X_train)
        self.allsentytrain = sparse.csr_matrix(hstack([axx,senty]))
        
        
        self.y_train = np.array(dfTrain["spoiler"], dtype=int)
        
        self.X_val = self.get_test_features(list(dfTrain["combined"][-500:]),list(dfTrain["trope"][-500:]),list(gen[-500:]))
        axxval =coo_matrix(self.X_val)
        self.allsentyval = sparse.csr_matrix(hstack([axxval,senty[-500:]]))
        self.y_val = np.array(dfTrain["spoiler"][-500:], dtype=int)
        
        # train logistic regression model.  !!You MAY NOT CHANGE THIS!! 
        self.logreg =  LogisticRegression(C=1.0,random_state=random_state,penalty='l2')
        self.logreg.fit(self.allsentytrain, self.y_train)
        
        
    def model_predict(self):
        """
        Method to read in test data from file, make predictions
        using trained model, and dump results to file 
        """
        
        # read in test data 
        dfTest  = pd.read_csv("../data/spoilers/test.csv")
        
        # featurize test data 
        dfTest["combined"] = dfTest["sentence"]
        
        
        senty = np.array([0,0,0,0,0,0])
        for idx,sentence in enumerate(dfTest["combined"]):
            ss = sid.polarity_scores(sentence)
            temp = np.array([ss['compound'],ss['neg'], ss['neu'], ss['pos'], float(sum(1 for c in sentence if c.isupper()))/float(len(sentence)),float(rt_test[idx])])
            senty = np.vstack((senty,temp))
        senty = senty[1:]
        
        self.X_test = self.get_test_features(list(dfTest["combined"]),list(dfTest["trope"]),list(gen_test))
        
        axx = coo_matrix(self.X_test)
        self.allsentytest = sparse.csr_matrix(hstack([axx,senty]).toarray())
        
        val_pred = self.logreg.predict(self.allsentyval)
        # make predictions on test data 
        pred = self.logreg.predict(self.allsentytest)

        # dump predictions to file for submission to Kaggle  
        pd.DataFrame({"spoiler": np.array(pred, dtype=bool)}).to_csv("prediction.csv", index=True, index_label="Id")
        #return val_pred

In [83]:
# Instantiate the FeatEngr clas 
feat = FeatEngr()

# Train your Logistic Regression classifier 
feat.train_model(random_state=1230)

# Shows the top 10 features for each class 
#feat.show_top10()

# Make prediction on test data and produce Kaggle submission file 
#val_pred = feat.model_predict()
feat.model_predict()

In [79]:
#print(len(val_pred))
#len(feat.y_val)
#val_pred = feat.model_predict()
np.sum(np.equal(val_pred,feat.y_val))/len(feat.y_val)
#val_pred
#feat.y_val

0.73799999999999999

In [7]:
feat.y_val

array([0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0,

### [25 points] Problem 2: Motivation and Analysis 
***

The job of the written portion of the homework is to convince the grader that:

- Your new features work
- You understand what the new features are doing
- You had a clear methodology for incorporating the new features

Make sure that you have examples and quantitative evidence that your features are working well. Be sure to explain how you used the data (e.g., did you have a validation set? did you do cross-validation?) and how you inspected the results. In addition, it is very important that you show some kind of an **error analysis** throughout your process.  That is, you should demonstrate that you've looked at misclassified examples and put thought into how you can craft new features to improve your model. 

A sure way of getting a low grade is simply listing what you tried and reporting the Kaggle score for each. You are expected to pay more attention to what is going on with the data and take a data-driven approach to feature engineering.

### Hints 
***

- Don't use all the data until you're ready. 

- Examine the features that are being used.

- Do error analyses.

- If you have questions that aren’t answered in this list, feel free to ask them on Piazza.

### FAQs 
***

> Can I heavily modify the FeatEngr class? 

Totally.  This was just a starting point.  The only thing you cannot modify is the LogisticRegression classifier.  

> Can I look at TV Tropes?

In order to gain insight about the data yes, however, your feature extraction cannot use any additional data (beyond what I've given you) from the TV Tropes webpage.

> Can I use IMDB, Wikipedia, or a dictionary?

Yes, but you are not required to. So long as your features are fully automated, they can use any dataset other than TV Tropes. Be careful, however, that your dataset does not somehow include TV Tropes (e.g. using all webpages indexed by Google will likely include TV Tropes).

> Can I combine features?

Yes, and you probably should. This will likely be quite effective.

> Can I use Mechanical Turk?

That is not fully automatic, so no. You should be able to run your feature extraction without any human intervention. If you want to collect data from Mechanical Turk to train a classifier that you can then use to generate your features, that is fine. (But that’s way too much work for this assignment.)

> Can I use a Neural Network to automatically generate derived features? 

No. This assignment is about your ability to extract meaningful features from the data using your own experimentation and experience.

> What sort of improvement is “good” or “enough”?

If you have 10-15% improvement over the baseline (on the Public Leaderboard) with your features, that’s more than sufficient. If you fail to get that improvement but have tried reasonable features, that satisfies the requirements of assignment. However, the extra credit for “winning” the class competition depends on the performance of other students.

> Where do I start?  

It might be a good idea to look at the in-class notebook associated with the Feature Engineering lecture where we did similar experiments. 


> Can I use late days on this assignment? 

You can use late days for the write-up submission, but the Kaggle competition closes at **4:59pm on Friday February 23rd**

> Why does it say that the competition ends at 11:59pm when the assignment says 4:59pm? 

The end time/date are in UTC.  11:59pm UTC is equivalent to 4:59pm MST.  Kaggle In-Class does not allow us to change this. 